# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 3:32PM,242802,12,8326080,Hush Hush,Released
1,Jul 26 2022 3:32PM,242801,19,ACG-2102490725,ACG North America LLC,Released
2,Jul 26 2022 3:31PM,242799,20,8204222,"ACI Healthcare USA, Inc.",Released
3,Jul 26 2022 3:28PM,242798,19,60016009-T1,"GUSA Granules USA, Inc.",Released
4,Jul 26 2022 3:28PM,242798,19,60016009-T2,"GUSA Granules USA, Inc.",Released
5,Jul 26 2022 3:21PM,242796,19,60016875,"GCH Granules USA, Inc.",Released
6,Jul 26 2022 3:13PM,242795,19,60016526,"GCH Granules USA, Inc.",Released
7,Jul 26 2022 3:13PM,242793,10,MSP212560,"Methapharm, Inc.",Released
8,Jul 26 2022 3:13PM,242793,10,MSP212561,"Methapharm, Inc.",Released
9,Jul 26 2022 3:13PM,242793,10,MSP212562,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,242796,Released,1
43,242798,Released,2
44,242799,Released,1
45,242801,Released,1
46,242802,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242796,NaN,NaN,1.0
242798,NaN,NaN,2.0
242799,NaN,NaN,1.0
242801,NaN,NaN,1.0
242802,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242702,0.0,1.0,0.0
242718,11.0,26.0,2.0
242720,0.0,0.0,50.0
242721,11.0,3.0,0.0
242723,0.0,5.0,26.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242702,0,1,0
242718,11,26,2
242720,0,0,50
242721,11,3,0
242723,0,5,26


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242702,0,1,0
1,242718,11,26,2
2,242720,0,0,50
3,242721,11,3,0
4,242723,0,5,26


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242702,,1,
1,242718,11,26,2
2,242720,,,50
3,242721,11,3,
4,242723,,5,26


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 3:32PM,242802,12,Hush Hush
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc."
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc."
5,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc."
6,Jul 26 2022 3:13PM,242795,19,"GCH Granules USA, Inc."
7,Jul 26 2022 3:13PM,242793,10,"Methapharm, Inc."
10,Jul 26 2022 3:12PM,242794,10,"Methapharm, Inc."
11,Jul 26 2022 3:07PM,242791,19,"GCH Granules USA, Inc."
12,Jul 26 2022 3:07PM,242788,19,Eli Lilly and Company


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 3:32PM,242802,12,Hush Hush,,,1
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC,,,1
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc.",,,1
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc.",,,2
4,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc.",,,1
5,Jul 26 2022 3:13PM,242795,19,"GCH Granules USA, Inc.",,,1
6,Jul 26 2022 3:13PM,242793,10,"Methapharm, Inc.",,,3
7,Jul 26 2022 3:12PM,242794,10,"Methapharm, Inc.",,,1
8,Jul 26 2022 3:07PM,242791,19,"GCH Granules USA, Inc.",,,1
9,Jul 26 2022 3:07PM,242788,19,Eli Lilly and Company,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 3:32PM,242802,12,Hush Hush,1,,
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC,1,,
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc.",1,,
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc.",2,,
4,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc.",1,,
5,Jul 26 2022 3:13PM,242795,19,"GCH Granules USA, Inc.",1,,
6,Jul 26 2022 3:13PM,242793,10,"Methapharm, Inc.",3,,
7,Jul 26 2022 3:12PM,242794,10,"Methapharm, Inc.",1,,
8,Jul 26 2022 3:07PM,242791,19,"GCH Granules USA, Inc.",1,,
9,Jul 26 2022 3:07PM,242788,19,Eli Lilly and Company,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 3:32PM,242802,12,Hush Hush,1,,
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC,1,,
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc.",1,,
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc.",2,,
4,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 3:32PM,242802,12,Hush Hush,1.0,NaN,NaN
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC,1.0,NaN,NaN
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
4,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 3:32PM,242802,12,Hush Hush,1.0,0.0,0.0
1,Jul 26 2022 3:32PM,242801,19,ACG North America LLC,1.0,0.0,0.0
2,Jul 26 2022 3:31PM,242799,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
3,Jul 26 2022 3:28PM,242798,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
4,Jul 26 2022 3:21PM,242796,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2913317,35.0,34.0,12.0
12,242802,1.0,0.0,0.0
15,485510,2.0,0.0,1.0
16,728144,0.0,7.0,22.0
19,2427912,13.0,0.0,0.0
20,1942008,117.0,62.0,11.0
22,485521,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2913317,35.0,34.0,12.0
1,12,242802,1.0,0.0,0.0
2,15,485510,2.0,0.0,1.0
3,16,728144,0.0,7.0,22.0
4,19,2427912,13.0,0.0,0.0
5,20,1942008,117.0,62.0,11.0
6,22,485521,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,34.0,12.0
1,12,1.0,0.0,0.0
2,15,2.0,0.0,1.0
3,16,0.0,7.0,22.0
4,19,13.0,0.0,0.0
5,20,117.0,62.0,11.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,12,Released,1.0
2,15,Released,2.0
3,16,Released,0.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,12.0,0.0,1.0,22.0,0.0,11.0,0.0
Executing,34.0,0.0,0.0,7.0,0.0,62.0,0.0
Released,35.0,1.0,2.0,0.0,13.0,117.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,12.0,0.0,1.0,22.0,0.0,11.0,0.0
1,Executing,34.0,0.0,0.0,7.0,0.0,62.0,0.0
2,Released,35.0,1.0,2.0,0.0,13.0,117.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,12.0,0.0,1.0,22.0,0.0,11.0,0.0
1,Executing,34.0,0.0,0.0,7.0,0.0,62.0,0.0
2,Released,35.0,1.0,2.0,0.0,13.0,117.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()